In [1]:
import pandas as pd
import requests
#import piexif - Do not need PIEXIF
from google.cloud import vision
from google.cloud.vision import types
from google.oauth2 import service_account
import os
import io
import json
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from bs4 import BeautifulSoup

In [21]:
FOLDER = '/Documents/Uploads'
APIKEY = pd.read_csv('/Users/anshul/Documents/APIKEYS/google_api_key.csv')
GOOGLE_SERVICE = '/Users/anshul/Downloads/My Project-93e8fa84a8bb.json'
N_ADDRESS = 4
trial_photo_location = 'Assets/trial.jpg'
new_columns = ['GPSLongitude', 'GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef']
new_columns_2 = ['4', '1', '2', '3']

#Instantiating Flask:
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = FOLDER

#Open an Image:
def get_exif_data(img):
    exif = Image.open(img)._getexif()
    if exif is not None:
        for key, value in exif.items():
            name = TAGS.get(key, key)
            exif[name] = exif.pop(key)
    else:
        return print("No GPS/Location Data Exists!")
            
    if 'GPSInfo' in exif: #Exif/GPS data is lost when an image is uploaded to Slack, we NEED RAW data.
        for key in exif['GPSInfo'].keys():
            name = GPSTAGS.get(key, key)
            exif['GPSInfo'][name] = exif['GPSInfo'].pop(key)
            
        exif_df = pd.DataFrame.from_dict(exif['GPSInfo'], orient='index') # Save to DataFrame from Dictionary
        exif_df = exif_df.T[new_columns]
        exif_datetime = pd.DataFrame.from_dict(exif, orient='index') # Isolate DateTimeOriginal from original exif dictionary
        exif_df['DateTimeOriginal'] = exif_datetime.T['DateTimeOriginal'] # Add new column to exif_df with DateTimeOriginal
        
        return exif_df    

    if '34853' in exif: #Exif data from Android phones
        for key in exif['34853'].keys():
            name = GPSTAGS.get(key, key)
            exif['34853'][name] = exif['34853'].pop(key)
            
        exif_df_android = pd.DataFrame.from_dict(exif['34853'], orient='index') # Save to DataFrame from Dictionary
        exif_df_android = exif_df_android.T[new_columns]
        exif_datetime_android = pd.DataFrame.from_dict(exif, orient='index') # Isolate DateTimeOriginal from original exif dictionary
        exif_df_android['DateTimeOriginal'] = exif_datetime_android.T['DateTimeOriginal'] # Add new column to exif_df with DateTimeOriginal
        
        return exif_df_android
       

    #return exix

In [23]:
get_exif_data('Assets/trial2.jpg')

NameError: name 'exif_df_android' is not defined